In [ ]:
import pandas as pd

# Basis pengetahuan hama dan nilai CF tiap gejala
KNOWLEDGE_BASE = {
    'Hama Wereng': {'daun_menguning': 0.8, 'tanaman_kerdil': 0.7, 'hopperburn': 0.9},
    'Hama Tikus': {'batang_putus': 0.95, 'tanaman_roboh': 0.9, 'bekas_gigitan': 1.0},
    'Hama Ulat': {'daun_berlubang': 0.85, 'kotoran_ulat': 0.9, 'daun_menggulung': 0.75}
}

# Daftar semua gejala unik dari basis pengetahuan
ALL_SYMPTOMS = list(set(gejala for hama in KNOWLEDGE_BASE.values() for gejala in hama))

# Menghitung CF(H,E) berdasarkan CF aturan dan CF user
def hitung_cf_gejala(cf_h, cf_e):
    return cf_h * cf_e

# Menggabungkan dua nilai CF (multiple evidence)
def gabung_cf(cf_old, cf_new):
    return cf_old + cf_new * (1 - cf_old)

# Melakukan diagnosis semua hama berdasarkan gejala input user
def diagnosis_cf(input_gejala):
    hasil_diagnosis = {}
    for hama, gejala_hama in KNOWLEDGE_BASE.items():
        cf_hama = 0.0
        for gejala, cf_h in gejala_hama.items():
            if gejala in input_gejala and input_gejala[gejala] > 0:
                cf_e = input_gejala[gejala]
                cf_baru = hitung_cf_gejala(cf_h, cf_e)
                cf_hama = cf_baru if cf_hama == 0.0 else gabung_cf(cf_hama, cf_baru)
        hasil_diagnosis[hama] = round(cf_hama, 4)
    return hasil_diagnosis

# Mengambil input CF user untuk setiap gejala
def get_user_input():
    print("--- Input Keyakinan Petani (CF User) ---")
    print("Skala: 0.0 (Tidak Yakin) - 1.0 (Sangat Yakin)")
    input_user = {}
    ALL_SYMPTOMS.sort()
    for gejala in ALL_SYMPTOMS:
        while True:
            try:
                cf_user = input(f"CF '{gejala.replace('_', ' ')}': ")
                cf_user = 0.0 if cf_user == '' else float(cf_user)
                if 0.0 <= cf_user <= 1.0:
                    input_user[gejala] = cf_user
                    break
                else:
                    print("Nilai harus antara 0.0 - 1.0.")
            except ValueError:
                print("Masukkan angka (contoh: 0.8).")
    return input_user

# Menampilkan hasil diagnosis, CF tertinggi, dan rekomendasi tindakan
def tampilkan_output(hasil_diagnosis):
    if not hasil_diagnosis or max(hasil_diagnosis.values()) == 0.0:
        diagnosis_tertinggi, cf_tertinggi = "Tidak dapat didiagnosis (CF = 0.0)", 0.0
    else:
        cf_tertinggi = max(hasil_diagnosis.values())
        diagnosis_tertinggi = max(hasil_diagnosis, key=hasil_diagnosis.get)

    print("\n" + "="*50)
    print("            HASIL DIAGNOSIS CERTAINTY FACTOR")
    print("="*50)

    df = pd.DataFrame(list(hasil_diagnosis.items()), columns=['Hama', 'Certainty Factor (CF)'])
    df['Persentase Keyakinan'] = (df['Certainty Factor (CF)'] * 100).round(2).astype(str) + " %"
    print(df.to_string(index=False))

    print("-" * 50)
    print(f"DIAGNOSIS TERTINGGI: {diagnosis_tertinggi}")
    print(f"Keyakinan: {cf_tertinggi} ({(cf_tertinggi * 100):.2f} %)")
    print("-" * 50)

    REKOMENDASI_BATAS = 0.6
    print("\nREKOMENDASI TINDAKAN:")
    if cf_tertinggi >= REKOMENDASI_BATAS:
        print(f"✅ Keyakinan ({cf_tertinggi}) ≥ {REKOMENDASI_BATAS}, lakukan pengendalian untuk {diagnosis_tertinggi}.")
    elif cf_tertinggi > 0:
        print(f"⚠️ Keyakinan ({cf_tertinggi}) < {REKOMENDASI_BATAS}, lakukan pengamatan lanjutan.")
    else:
        print("ℹ️ Tidak ada indikasi hama signifikan, tanaman aman.")
    print("="*50)

# Program utama: ambil input, hitung diagnosis, dan tampilkan hasil
if __name__ == "__main__":
    cf_user_input = get_user_input()
    cf_user_input = {k: v for k, v in cf_user_input.items() if v > 0.0}
    if not cf_user_input:
        print("\nTidak ada gejala yang dilaporkan. Diagnosis tidak dilakukan.")
    else:
        hasil = diagnosis_cf(cf_user_input)
        tampilkan_output(hasil)
